In [1]:
import torch
import spacy
from nltk.corpus import wordnet as wn
import numpy as np
import re

In [2]:
class TextSimilarity:
    
    def __init__(self, tags_dict = None, correlation_matrix = None):
        
        self._tags = self._get_tags_dict() if tags_dict is None else tags_dict
        self._no_of_tags = len(self._tags) 
        self._tag_correlation_matrix = np.identity(self._no_of_tags) if correlation_matrix is None else correlation_matrix
        self._parser = spacy.load("en")
        
    def _get_tags_dict(self):
        with open("data/tags.txt","r") as fl:
            data = fl.read()

        tags = [each.strip("\n").strip() for each in re.findall(r"\n[a-z]*\s",data)]
        tags.extend(["acl","ROOT"])
        tags = sorted(tags)
        tags = {each[1]:each[0] for each in enumerate(tags,start=0)}
        return tags
    
    def _similarity_word(self, pair_A, pair_B):

        #getting head and dependent texts 
        head_a, head_b = pair_A[0].text, pair_B[0].text
        dep_a, dep_b = pair_A[2].text, pair_B[2].text

        if head_a == head_b:
            head = 1
        else:
            try:
                #WordNet synsets for heads
                head_a, head_b = wn.synsets(head_a)[0], wn.synsets(head_b)[0]

                #path based similarity (Li et. al) for head
                head = head_a.path_similarity(head_b)

                head = 0 if head is None else head  

            except Exception:
                head = 0

        if dep_a == dep_b:
            dep = 1
        else:
            try:
                #WordNet synsets for dependent
                dep_a, dep_b = wn.synsets(dep_a)[0], wn.synsets(dep_b)[0]

                #path based similarity (Li et. al) for dependent
                dep = dep_a.path_similarity(dep_b)

                dep = 0 if dep is None else dep

            except Exception:
                dep = 0     

        return head + dep

    def _similarity_tag(self, tag_a, tag_b):
        
        tag_a_id, tag_b_id = self._tags[tag_a], self._tags[tag_b] 
        score = self._tag_correlation_matrix[tag_a_id,tag_b_id]
        
        return score

    def semantic_similarity(self, document_1, document_2):

        #parsing documets using spaCy English language parser
        tokens_1,tokens_2 = self._parser(document_1), self._parser(document_2)

        #seperating dependency pairs and tags from tokens
        pairs_1 = [(token.head,token.dep_,token) for token in tokens_1]
        pairs_2 = [(token.head,token.dep_,token) for token in tokens_2]

        score = 0

        #calculating score 
        for pair_A in pairs_1:

            for pair_B in pairs_2:

                score += self._similarity_word(pair_A, pair_B) * self._similarity_tag(pair_A[1], pair_B[1])

        #averaging score 
        score = score / (len(tokens_1) + len(tokens_2))

        return score 

In [11]:
sim = TextSimilarity()
sim._tag_correlation_matrix = torch.from_numpy(sim._tag_correlation_matrix)
sim._tag_correlation_matrix.requires_grad = True
score = sim.semantic_similarity("This is a boy.","This is a girl.")

In [12]:
def loss_function(score, target):
    return target - score

In [16]:
loss = loss_function(score, 0.9)
loss.backward()
sim._tag_correlation_matrix

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]],
       dtype=torch.float64, requires_grad=True)

In [ ]:
def train(data, model, epochs = 25, lr = 0.001):
    
    model._tag_correlation_matrix = torch.from_numpy(model._tags)
    model._tag_correlation_matrix.requires_grad = True
    
    criterion = loss_function
    optimizer = torch.optim.Adam(model._tags, lr=lr)
    
    